# Different data structures

In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
import jupyter_black

jupyter_black.load()

# Project
from src.data_connectors import read_input_files
from src.data_preparation import search_space
from src.data_connectors.read_input_files import Instance

In [9]:
instance = 333
instances_path = "../data/input/HRTInstances"
ins_x = read_input_files.read_file(
    os.path.join(instances_path, f"Instance_{instance}.txt")
)

In [10]:
ins_x.df_setup

,Humans,Robots,WorkingSpaces
0,2,3,3


In [11]:
ins_x.df_resources

,Type,Id
0,H,0
1,H,1
2,R,2
3,R,3
4,R,4


In [12]:
ins_x.df_workingspace_resources

,WorkingSpace,Resource
0,1,2
1,1,0
2,1,1
3,2,3
4,2,0
5,2,1
6,3,4
7,3,0
8,3,1


In [13]:
ins_x.df_workingspace_id.head()

,WorkingSpace,Id
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


## [Flow](https://miro.com/app/board/uXjVPyl00iw=/)

1. Generate **Initial Population**
2. Verify if order of genes is violating the precedence
    1. Discard those that are violating the precedence
3. Create the array of times (start and end), using the resources time estimator for the respective mode allocation
4. Duplicate it for all the jobs in the same instance (SAC)
5. **Fitness Function**: Compute the total time to complete all the jobs in all working spaces of same instance $\Longrightarrow C$ 
6. **Selection**: Retain the genes with lower C (at maximum X)
    1. Discard those that have larger C
    2. Verify if new population has a significant improvement in total C from previous population.
7. **Generate next population**:
    1. **Crossover**: from survival chromosomes, create offsprings 
    2. **Mutation**: add mutation to the created offsprings (initially larger %, and decreases over time).
8. Return to point **2**.

### 0. The chromosome

```python
chromosome = {
    'mode': [0, 1, 2, 2, 1, ... ], # resource ID
    'order': [2, 3, 1, 4, 5, ...], # order of tasks of each position
    }

```

In [14]:
from src.genetic_algorithm.chromosome import Chromosome

### 1. Generate initial population

By default, it generates population for 1 working space:

**Note**:
- *It is important to generate a large first population, because a lot chromosomes might not survived due to feasibility of solutions.*

In [15]:
from src.genetic_algorithm import first_population

# Modes

possible_modes = first_population.get_possible_modes(ins_x)
# Tasks
n_tasks = first_population.get_total_number_of_tasks_per_working_space(ins_x)

# Chromosomes
chromosomes = first_population.get_first_population(possible_modes, n_tasks)

Possible modes: ['0' '1' '2' '0-2' '1-2']
Number of tasks: 20
Total size of first population: 6600


### 2. Verify feasibility of solutions

We need to verify:
- the precedence is valid
- the combination of modes and tasks is possible (the selected mode can perform the associated task) 

In [102]:
from src.genetic_algorithm import feasibility

In [103]:
ins_x.df_predecessor_sucessor.head()

,Predecessor,Sucessor
0,1,2
1,2,3
2,3,4
3,1,4
4,1,5


In [104]:
precedence_feasible_chromosomes = []
for i in range(len(chromosomes)):
    chrom = chromosomes[i]
    if feasibility.is_chromosome_precedence_feasible(ins_x, chrom):
        precedence_feasible_chromosomes.append(chrom)

print(f"From {len(chromosomes)} to {len(precedence_feasible_chromosomes)}")

From 6600 to 110


In [105]:
task_mode_feasible_chromosomes = []
for i in range(len(precedence_feasible_chromosomes)):
    chrom = precedence_feasible_chromosomes[i]
    if feasibility.is_chromosome_task_mode_feasible(ins_x, chrom):
        task_mode_feasible_chromosomes.append(chrom)

print(
    f"From {len(precedence_feasible_chromosomes)} to {len(task_mode_feasible_chromosomes)}"
)

From 110 to 110
